In [8]:
import json
import http.client
import pandas as pd
# from airflow.decorators import task
from cfg import DATA_PATH, URA_ACCESS_KEY
import requests

def get_token(access_key): 
    conn = http.client.HTTPSConnection("www.ura.gov.sg")
    payload = ''
    headers = {
    'AccessKey': access_key
    }
    conn.request("GET", "/uraDataService/insertNewToken.action", payload, headers)
    res = conn.getresponse()
    data = res.read()
    return json.loads(data.decode("utf-8"))['Result']
    # res = requests.get("https://www.ura.gov.sg/uraDataService/insertNewToken.action", headers=headers)
    # return res.json()['Result']


def get_result(token, access_key, route):
    headers = {
        "AccessKey": access_key,
        "Token": token,
        "User-Agent": "PostmanRuntime/7.26.8",
    }
    res = requests.get("https://www.ura.gov.sg" + route, headers=headers)
    data = res.json()['Result']
    return data

def get_all_results(token, access_key, routes):
    result = []
    for route in routes:
        result.extend(get_result(token, access_key, route))
    return result

def get_all_ura():
    private_transactions_routes = [
        "/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=" + str(i)
        for i in range(1, 5)
    ]

    access_key = URA_ACCESS_KEY
    token = get_token(access_key)

    df_private_transactions = pd.json_normalize(
        data = get_all_results(token, access_key, private_transactions_routes), 
        record_path='transaction', 
        meta = ['street', 'project', 'marketSegment']
        )

    return df_private_transactions


# @task
def extract_ura_data_task():
    print("Getting URA data...")
    df_private_transactions = get_all_ura()
    print("URA data obtained.")

    data_path_private_transactions = DATA_PATH + "/private_transactions.csv"

    # save to csv
    print("Saving to csv...")
    df_private_transactions.to_csv(data_path_private_transactions, index=False)
    print("done")
    return data_path_private_transactions

In [9]:
extract_ura_data_task()

Getting URA data...
URA data obtained.
Saving to csv...


OSError: Cannot save file into a non-existent directory: '/opt/airflow/dags/data'

In [15]:
def get_cpi():
    urlData = "https://tablebuilder.singstat.gov.sg/api/table/tabledata/M212881"
    headers = {
        'User-Agent': 'PostmanRuntime/7.26.8',
        'Accept': '*/*',
        'Cache-Control': 'no-cache',
        'Host': 'tablebuilder.singstat.gov.sg',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Content-Type': 'application/json',
    }
    # result = requests.get(urlData).json()
    # result = result['Data']
    res = requests.get(urlData, headers=headers)
    result = res.json()['Data']

    key_value_pairs = result['row'][0]['columns']
    # turn into a dataframe
    df = pd.DataFrame(key_value_pairs)
    # rename columns 
    df.columns = ['year_month', 'Value']
    # convert to datetime
    df['year_month'] = pd.to_datetime(df['year_month'], format='%Y %b')
    # rename columns
    df.columns = ['Month', 'Value']
    return df


def extract_cpi_task():
    print("Getting Singstat data...")
    data_path = "/opt/airflow/dags/data"
    df_cpi = get_cpi()
    data_path_cpi = data_path + "/cpi.csv"
    df_cpi.to_csv(data_path_cpi, index=False)
    print("Singstat data obtained.")
    return data_path_cpi

get_cpi()

{'id': 'M212881', 'title': 'Consumer Price Index (CPI), 2019 As Base Year, Monthly', 'footnote': "The weighting pattern of the Consumer Price Index (CPI) was derived from the expenditure values collected from the 2017/18 Household Expenditure Survey (HES) and updated to 2019 values by taking into account price changes between 2017/18 and 2019.  For more information on the CPI rebasing, refer to the Information Paper 'Rebasing of the Consumer Price Index (2019 as Base Year)' (https://www.singstat.gov.sg/-/media/files/publications/economy/ip-e48.pdf).", 'frequency': 'Monthly', 'datasource': 'SINGAPORE DEPARTMENT OF STATISTICS', 'generatedBy': 'SingStat Table Builder', 'dataLastUpdated': '25/03/2024', 'dateGenerated': '26/03/2024', 'offset': None, 'limit': '3000', 'sortBy': None, 'timeFilter': None, 'between': None, 'search': None, 'row': [{'seriesNo': '1', 'rowText': 'All Items', 'uoM': 'Index', 'footnote': '', 'columns': [{'key': '1961 Jan', 'value': '24.542'}, {'key': '1961 Feb', 'valu

,Month,Value
0,1961-01-01,24.542
1,1961-02-01,24.565
2,1961-03-01,24.585
3,1961-04-01,24.187
4,1961-05-01,24.053
...,...,...
753,2023-10-01,115.111
754,2023-11-01,114.91
755,2023-12-01,115.343
756,2024-01-01,114.572


In [34]:
# ! pip install openpyxl
df = pd.read_excel("data/districts.xlsx")

df.head()

,Postal District,Postal Sector,General Location
0,1,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People’s Park"
1,2,"07, 08","Anson, Tanjong Pagar"
2,3,"14, 15, 16","Queenstown, Tiong Bahru"
3,4,"09, 10","Telok Blangah, Harbourfront"
4,5,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."


In [33]:
# df['Postal Sector'].str.split(",").explode().str.strip()
df.reset_index(drop=True)

,Postal District,Postal Sector,General Location
0,1,"01, 02, 03, 04, 05, 06","Raffles Place, Cecil, Marina, People’s Park"
1,2,"07, 08","Anson, Tanjong Pagar"
2,3,"14, 15, 16","Queenstown, Tiong Bahru"
3,4,"09, 10","Telok Blangah, Harbourfront"
4,5,"11, 12, 13","Pasir Panjang, Hong Leong Garden, Clementi New..."
5,6,"17,","High Street, Beach Road (part)"
6,7,"18, 19","Middle Road, Golden Mile"
7,8,"20, 21",Little India
8,9,"22, 23","Orchard, Cairnhill, River Valley"
9,10,"24, 25, 26, 27","Ardmore, Bukit Timah, Holland Road, Tanglin"


In [35]:
def transform_districts(districts: pd.DataFrame):
    districts = districts.reset_index()
    districts["Postal Sector"] = districts["Postal Sector"].str.split(",").explode()
    districts = districts.reset_index()
    districts["Postal Sector"] = districts["Postal Sector"].str.strip()
    districts.drop(columns=["General Location"], inplace=True)
    districts.drop_duplicates(inplace=True)
    districts = districts[districts["Postal Sector"].str.len() == 2]
    return districts

transform_districts(df)

ValueError: cannot reindex on an axis with duplicate labels